In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from itertools import product

In [ ]:
ticker = "JPM"
data = yf.download(tickers = ticker, start="2006-01-01", end="2025-01-01")

We mainly use the conditional probabilities formula
$$ P(A|B) = \frac{P(A \cap B)}{P(B)}.$$
We define sets $A_{n, \varepsilon}$ which will denote an increase (or decrease) of $\varepsilon$ percent over $n$ days. We will iterate over $\varepsilon, n$ and the possible combinations of up and downs before. In a first step we will calculate the sets $A_{n, \varepsilon}$.

In [3]:
data["Daily return"] = data["Adj Close"].pct_change() #periods=n
data["State"] = np.where(data["Daily return"] >= 0, "up", "down")

data = data.drop(["Daily return"], axis=1)

In [ ]:
states = ["up", "down"]
transition_probabilities = {}
epsilon_prev = 0

for length in range(2, 11): 

    for n_days in range(1, 10):
        data[f"change_{n_days}"] = data["Adj Close"].pct_change(n_days)
        for epsilon in np.arange(0.1, 5, 0.1):
            condition1 = data[(data[f"change_{n_days}"] >= epsilon_prev) & (data[f"change_{n_days}"] < epsilon)]
            epsilon_prev = epsilon

            for state_comb in product(states, repeat=length):
                condition2 = True
                for i, state in enumerate(state_comb):
                    # In our case it does not matter if we look forward (i.e. -i) or backwards (i.e. i) because in any case we iterate over all combinations.
                    condition1 &= (data["State"].shift(i) == state)
                    # The first state is the one we "predict"
                    if i != 0:
                        condition2 &= (data["State"].shift(i) == state)

                transition_probabilities[state_comb] = (len(data[condition1])/len(data[condition2]), len(data[condition2]))

In [ ]:
for state_comb, prob in transition_probabilities.items():
    if prob[0] > 0.7 and prob[1] > 10:
        print(f"Probability of {state_comb[0]} after {state_comb[1:]} is {prob[0]}, pattern appeared {prob[1]} times")